<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Проект по Sprint 0:</h2>

----

### Вариант задания №10


<h2 style="color:DodgerBlue">Описание проекта:</h2>
<h2 style="color:DodgerBlue">Описание задачи:</h2>

----


Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.

Требования к базовому классу Invoice:

• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).
• Методы:
o
o CalculateTotal(): метод для расчета общей суммы по фактуре.
o AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
o RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.

Требования к производным классам:

1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [3]:
using System;


public class LineItem
{
    public string Description { get; set; }
    public decimal UnitPrice { get; set; }
    public int Quantity { get; set; }
    public decimal Amount => UnitPrice * Quantity;

    public LineItem(string description, decimal unitPrice, int quantity)
    {
        Description = description;
        UnitPrice = unitPrice;
        Quantity = quantity;
    }
}

public class Invoice
{
    public string InvoiceNumber { get; set; }
    public DateTime IssueDate { get; set; }
    public decimal TotalAmount { get; protected set; }
    protected List<LineItem> LineItems { get; set; }

    public Invoice(string invoiceNumber, DateTime issueDate)
    {
        InvoiceNumber = invoiceNumber;
        IssueDate = issueDate;
        LineItems = new List<LineItem>();
        TotalAmount = 0;
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = LineItems.Sum(item => item.Amount);
    }

    public virtual void AddLine(LineItem lineItem)
    {
        LineItems.Add(lineItem);
        CalculateTotal();
    }

    public virtual void RemoveLine(LineItem lineItem)
    {
        LineItems.Remove(lineItem);
        CalculateTotal();
    }

    public virtual void PrintInfo()
    {
        Console.WriteLine($"Фактура #{InvoiceNumber}");
        Console.WriteLine($"Дата: {IssueDate:dd.MM.yyyy}");
        Console.WriteLine($"Сумма: {TotalAmount} руб.");
    }
}

public class GoodsInvoice : Invoice
{
    public DateTime SupplyDate { get; set; }

    public GoodsInvoice(string invoiceNumber, DateTime issueDate, DateTime supplyDate) 
        : base(invoiceNumber, issueDate)
    {
        SupplyDate = supplyDate;
    }

    public override void AddLine(LineItem lineItem)
    {
        base.AddLine(lineItem);
        Console.WriteLine($"Дата поставки: {SupplyDate:dd.MM.yyyy}");
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        Console.WriteLine($"Поставка: {SupplyDate:dd.MM.yyyy}");
    }
}

public class ServiceInvoice : Invoice
{
    public DateTime ServiceDate { get; set; }

    public ServiceInvoice(string invoiceNumber, DateTime issueDate, DateTime serviceDate) 
        : base(invoiceNumber, issueDate)
    {
        ServiceDate = serviceDate;
    }

    public override void RemoveLine(LineItem lineItem)
    {
        Console.Write("Причина удаления: ");
        string reason = Console.ReadLine();
        base.RemoveLine(lineItem);
        Console.WriteLine($"Услуга удалена. Причина: {reason}");
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        Console.WriteLine($"Услуга оказана: {ServiceDate:dd.MM.yyyy}");
    }
}

public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; set; }
    private decimal returnAmount;

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed) 
        : base(invoiceNumber, issueDate)
    {
        ReturnAllowed = returnAllowed;
        returnAmount = 0;
    }

    public void SetReturnAmount(decimal amount)
    {
        if (ReturnAllowed)
        {
            returnAmount = amount;
            CalculateTotal();
        }
    }

    public override void CalculateTotal()
    {
        TotalAmount = LineItems.Sum(item => item.Amount) - returnAmount;
    }

    public override void PrintInfo()
    {
        base.PrintInfo();
        if (ReturnAllowed && returnAmount > 0)
        {
            Console.WriteLine($"Возврат: {returnAmount} руб.");
        }
    }
}

class Program
{
    static void Main()
    {
        GoodsInvoice goods = new GoodsInvoice("G-001", DateTime.Now, DateTime.Now.AddDays(7));
        goods.AddLine(new LineItem("Ноутбук", 50000, 1));
        goods.AddLine(new LineItem("Мышь", 1000, 2));
        goods.PrintInfo();
        Console.WriteLine();

        ServiceInvoice service = new ServiceInvoice("S-001", DateTime.Now, DateTime.Now.AddDays(1));
        service.AddLine(new LineItem("Консультация", 5000, 2));
        service.PrintInfo();
        Console.WriteLine();

        CombinedInvoice combined = new CombinedInvoice("C-001", DateTime.Now, true);
        combined.AddLine(new LineItem("Товар", 10000, 1));
        combined.SetReturnAmount(2000);
        combined.PrintInfo();
    }
}